In [10]:
using CMBLensing
using PythonPlot


Cℓ  = camb(r=0.05, ℓmax=25000)         
Cℓn = noiseCℓs(μKarcminT=1, ℓknee=100)


θpix  = 0.7438046267475303        # pixel size in arcmin
Nside = 512      # number of pixels per side in the map
pol   = :I       # type of data to use (can be :T, :P, or :TP)
T     = Float32  # data type (Float32 is ~2 as fast as Float64);
bandpass_mask = LowPass(5000);

(;f, f̃, ϕ, ds) = load_sim(
    seed = 3,
    Cℓ = Cℓ,
    Cℓn = Cℓn,
    θpix = θpix,
    T = T,
    Nside = Nside,
    # beamFWHM = 1.4,
    pol = pol,
    bandpass_mask = bandpass_mask, 
    #pixel_mask_kwargs = (edge_padding_deg=0, apodization_deg=0, num_ptsrcs=0),
)

(f = ComplexF32[0.0f0 + 0.0f0im, -1.4355074f6 - 3.483249f6im, 3.2807038f6 - 4.531778f6im, -455595.7f0 + 120705.29f0im, -972343.9f0 + 297891.97f0im, 937233.5f0 + 1.9683731f6im, -106391.93f0 + 222771.42f0im, 379129.03f0 - 209760.14f0im, -269272.47f0 + 208467.47f0im, -43658.668f0 - 520942.03f0im  …  -2.9985697f0 + 1.7251345f0im, -10.190046f0 - 10.42925f0im, 0.3710811f0 + 3.7072842f0im, 5.331645f0 + 2.0137262f0im, -0.26298612f0 + 3.6719344f0im, 7.2763753f0 - 0.31616065f0im, -1.7443056f0 - 6.943406f0im, -11.064791f0 + 4.932134f0im, -5.0352793f0 + 7.488441f0im, 0.15712562f0 + 6.6466002f0im], f̃ = Float32[-149.3736, -147.7579, -145.56017, -143.00284, -140.87837, -139.30023, -138.73958, -138.59541, -138.75392, -138.56958  …  -87.92406, -99.09506, -109.467026, -118.06137, -124.803825, -130.9102, -136.25508, -139.98035, -142.26764, -143.0646], ϕ = ComplexF32[0.0f0 + 0.0f0im, 0.29005766f0 - 0.4476377f0im, 0.023752943f0 - 0.09520566f0im, 0.04078745f0 + 0.010236119f0im, -0.014798568f0 - 0.019218616

In [14]:
using NPZ

npzwrite("sim_maps.npz", Dict(
    # maps
    "f_unlensed" => f,
    "f_lensed"   => f̃,
    "phi"        => ϕ,
    "seed"  => 3,

))
println("Saved to sim_maps.npz")

Saved to sim_maps.npz


In [11]:
(; Cf, Cϕ) = ds


BaseDataSet(d, Cf, Cn, Cn̂, M, M̂, B, B̂, logprior, Cϕ, Cf̃, D, G, L, Nϕ)

In [13]:
using NPZ

# Extract covariance operators
(; Cf, Cϕ) = ds

npzwrite("sim_maps.npz", Dict(
    # maps
    "f_unlensed" => f,
    "f_lensed"   => f̃,
    "phi"        => ϕ,

    # ℓ and spectra (works for top-level CAMB output)
    "ell"        => Cℓ.total.TT.ℓ,
    "Cl_TT"      => Cℓ.total.TT.Cℓ,
    "Cl_phi"     => Cℓ.total.ϕϕ.Cℓ,

    # diagonal noise and prior covariances
    "Cln_diag"   => Cℓn.diag,   # FIXED 
    "Cf_diag"    => Cf.diag,    # unlensed prior
    "Cphi_diag"  => Cϕ.diag,    # φ prior

    # parameters
    "theta_pix_arcmin" => θpix,
    "Nside" => Nside,
    "seed"  => 3,
))

println("✅ Saved to sim_maps.npz")




LoadError: FieldError: type NamedTuple has no field `diag`, available fields: `TT`, `EE`, `BB`, `TE`

In [4]:
using NPZ

npzwrite("sim_maps.npz", Dict(
    # maps
    "f_unlensed" => f,
    "f_lensed"   => f̃,
    "phi"        => ϕ,

    # theory spectra (ARRAYS, not structs)
    "ell"        => collect(Cℓ.total.ϕϕ.ℓ),
    "Cl_phi"     => Cℓ.total.ϕϕ.Cℓ,
    "Cl_TT"      => Cℓ.total.TT.Cℓ,

    # noise power spectrum (also extract array)
    "Cln_ell"    => collect(Cℓn.ℓ),
    "Cln"        => Cℓn.Cℓ,

    # Covariance matrices used by MAP solver
    "Cf_diag"    => Cf.diag,    # Diagonal covariance
    "Cphi_diag"  => Cϕ.diag,

    # run params
    "theta_pix_arcmin" => θpix,
    "Nside" => Nside,
    "seed"  => 3,
))
println("Saved to sim_maps.npz")


LoadError: FieldError: type NamedTuple has no field `ℓ`, available fields: `TT`, `EE`, `BB`, `TE`

In [ ]:
ds = ds(;)

(;Cf, Cϕ) = ds

In [ ]:
using CMBLensing: Map, Loess

# inject loess into CMBLensing namespace
Base.eval(CMBLensing, :(loess = Loess.loess))

qe = quadratic_estimate(ds)

Ω0 = FieldTuple(ϕ = Map(qe.ϕqe))   # warm start φ from QE

result = MAP_joint(
    (;),            # θ = empty parameter block
    ds,
    Ω0;
    nsteps = 20,
    nburnin_update_hessian = 0,
    progress = true,
    conjgrad_kwargs = (tol=1e-3, nsteps=800),
)

fJ  = result.f
ϕJ  = result.ϕ
hist = result.history


In [ ]:

# ------------------------------------------------------------------
# 5) Compare spectra
# ------------------------------------------------------------------
ℓ = Cℓ.total.ℓ    # This prints the ell grid of the theoretical Cℓ
println("ℓ range in theory: ", extrema(ℓ))

loglog(ℓ.^4 .* Cℓ.total.ϕϕ, "k")                 # theory
loglog(get_ℓ⁴Cℓ(ϕ))                               # true phi
loglog(get_ℓ⁴Cℓ(ϕJ))                              # MAP reconstruction
loglog(get_ℓ⁴Cℓ(ϕJ - ϕ))                          # residual
loglog(get_ℓ⁴Cℓ(quadratic_estimate(ds).ϕqe))      # QE
xlim(100, 10000)
ylim(5e-12, 2e-6)
legend(["theory", raw"true $\phi$", raw"best-fit $\phi$",
        "difference", raw"QE $\phi$"])
xlabel(raw"$\ell$")
ylabel(raw"$\ell^4 C_\ell$")

PythonPlot.show()

# ------------------------------------------------------------------
# 5) Lensing map visualisation
# ------------------------------------------------------------------
PythonPlot.plot(
    1e6 .* [ϕ  ϕJ],
    title = ["true" "best-fit"] .* raw" $\phi$",
    vlim = 17
)


# ------------------------------------------------------------------
# 6) Correlation coefficient vs ell (ρℓ)
# ------------------------------------------------------------------
semilogx(get_ρℓ(ϕ, ϕJ))
semilogx(get_ρℓ(quadratic_estimate(ds).ϕqe, ϕ))
legend(["Joint MAP", "QE"])